<a href="https://colab.research.google.com/github/spiritofthegael/Automated-Labeling-Threat-Reports/blob/main/GloVe_tactics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras import backend as K
from keras import models
from keras import layers
from keras import regularizers
from google.colab import drive 

drive.mount('/content/gdrive')
#dataset of ~1400 threat reports labelled with MITRE ATT&CK labels
df = pd.read_csv('/content/gdrive/MyDrive/thesis_work/training_data_original.csv')
df.head()

Mounted at /content/gdrive


,Text,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,...,T1124,T1035,T1086,T1490,T1216,T1094,T1043,T1211,T1127,T1077
0,Talos Blog || Cisco Talos Intelligence Group -...,0,1,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
1,OilRig Actors Provide a Glimpse into Developme...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hogfish Redleaves Malware Threat Analysis I Ac...,1,1,0,1,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0
3,Scheduled Transfer - Enterprise | MITRE ATT&CK...,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Winlogon Helper DLL - Enterprise | MITRE ATT&C...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#restrict columns to just tactics
df = df.iloc[:, 0:13]
df.head()

,Text,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,TA0009,TA0011,TA0001
0,Talos Blog || Cisco Talos Intelligence Group -...,0,1,0,1,1,0,1,0,1,1,1,0
1,OilRig Actors Provide a Glimpse into Developme...,0,0,0,0,0,0,1,0,0,0,0,0
2,Hogfish Redleaves Malware Threat Analysis I Ac...,1,1,0,1,0,0,1,0,1,1,1,0
3,Scheduled Transfer - Enterprise | MITRE ATT&CK...,0,0,0,0,0,0,0,1,0,0,0,0
4,Winlogon Helper DLL - Enterprise | MITRE ATT&C...,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
import re
def reports_to_words(raw_report):
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z0-9]", " ", raw_report) 
    #
    # 3. Convert to lower case, split into individual words
    words = raw_report.lower().split()                             
    
    #
    # 7. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( words ))

In [ ]:
# Get the number of reports based on the dataframe column size
num_reports = len(df)

# Initialize an empty list to hold the clean exploits
clean_train_reports = []

# Loop over each exploit; create an index i that goes from 0 to the length
# of the exploit list
for i in range(0, num_reports):
    # Call our function for each one, and add the result to the list of
    # clean exploits
    clean_train_reports.append( reports_to_words(str(df['Text'][i])))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenize text

tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reports)
word_index = tokenizer.word_index
print(word_index)
vocab_size=len(word_index)
print(vocab_size)

# Padding data

sequences = tokenizer.texts_to_sequences(clean_train_reports)
padded = pad_sequences(sequences, maxlen=3000, padding='post', truncating='post')

{'the': 1, 'xe2': 2, 'to': 3, 'x80': 4, 'xc2': 5, 'and': 6, 'a': 7, 'of': 8, 'in': 9, 'is': 10, 'for': 11, 'that': 12, 'this': 13, 'on': 14, 'as': 15, 'with': 16, 'by': 17, 'file': 18, 'it': 19, 'com': 20, 'from': 21, '1': 22, 'are': 23, 'or': 24, 'be': 25, 'security': 26, 'an': 27, 'windows': 28, 'malware': 29, 'data': 30, 'system': 31, 'x9d': 32, '0': 33, '2': 34, 'used': 35, 'can': 36, 'exe': 37, 'we': 38, 'c': 39, 'not': 40, 'x99s': 41, 'use': 42, 'command': 43, 'you': 44, 'information': 45, 'microsoft': 46, 'if': 47, 'network': 48, 'server': 49, 'xa2': 50, 'user': 51, 'service': 52, 'at': 53, 'have': 54, 'all': 55, 'which': 56, '3': 57, 'files': 58, 'was': 59, '00': 60, 'using': 61, 'code': 62, 'has': 63, 'dll': 64, 'will': 65, 'name': 66, 'http': 67, 'may': 68, 'threat': 69, '4': 70, 'new': 71, 'access': 72, 'attack': 73, '5': 74, 'i': 75, 'retrieved': 76, 'process': 77, 'also': 78, 'domain': 79, 'software': 80, 'other': 81, 'more': 82, 'figure': 83, 'group': 84, 'www': 85, 'one'

In [ ]:
# Import tensor representations for words


embeddings_index = {};
#with open('/content/gdrive/MyDrive/content/gdrive/glove.42B.300d.txt') as f:
with open('/content/gdrive/MyDrive/content/gdrive/vectorsfull_300.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, 300));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

In [ ]:
def get_weighted_loss(weights):
  def weighted_loss(y_true, y_pred):
    return K.mean((weights[:,0]**(1-tf.cast(y_true, tf.float32)))*(weights[:,1]**(tf.cast(y_true, tf.float32)))
    *K.binary_crossentropy(tf.cast(y_true, tf.float32), y_pred, from_logits=True), axis=-1)
  return weighted_loss

In [ ]:
def create_lstm_model(class_weights, bidirectional=True):
    """
    generate Bi directional LSTM 
    Parameters:
    class_weights: for weighted loss function
    bidirectional: boolean value: to generate uni or bidirectional LSTM on top of CNN layers
    Return:
    the CNN -UniLSTM or CNN-BiLSTM model
    """
    lstm_model = models.Sequential()
    lstm_model.add(layers.Embedding(vocab_size+1, 300, weights=[embeddings_matrix], trainable=False))
    lstm_model.add(layers.Bidirectional(layers.LSTM(units=96, dropout=0.2, recurrent_dropout=0.2)))
    lstm_model.add(layers.Dense(48))
    lstm_model.add(layers.Dense(24))
    lstm_model.add(layers.Dense(12, activation='sigmoid'))
    lstm_model.compile(optimizer='adam', loss=get_weighted_loss(class_weights), metrics=['accuracy'])
    #lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return lstm_model

In [ ]:
padded

array([[ 2076,   180,  1585, ..., 60422,    81,   109],
       [ 1719,   428,   537, ...,     1,   496,  1909],
       [ 8805,  4005,    29, ...,     0,     0,     0],
       ...,
       [  985,   596, 31834, ...,     0,     0,     0],
       [ 1674,  1390,  1674, ...,  2698,   395,  6762],
       [ 1600,   575,   236, ...,     0,     0,     0]], dtype=int32)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

#set up five fold cross validation
kf = KFold(n_splits=5, random_state=42, shuffle=True)
recall = []
prec = []
#df = df.drop(['Text'], axis=1)
#for each fold
for train_index, test_index in kf.split(padded):
  #get train and test set
  X_train, X_test = padded[train_index], padded[test_index]
  y_train, y_test = np.array(df)[train_index], np.array(df)[test_index]

  all_labels = np.array(df)
  number_dim = np.shape(y_train)[1]
  print(number_dim)
  #compute weights for weighted loss function
  weights = np.empty([number_dim, 2])
  for i in range(number_dim):
    weights[i] = compute_class_weight(class_weight = 'balanced', classes = [0.,1.], y = all_labels[:, i])
  print("weights: ", weights)
  #create model
  lstm_model = create_lstm_model(weights)
  history = lstm_model.fit(X_train, y_train, batch_size=128, epochs = 5, validation_data=(X_test, y_test))  
  pred = lstm_model.predict(X_test)
  pred[pred>=.5] = 1
  pred[pred<.5] = 0
  recall.append(recall_score(y_test, pred, average=None))
  prec.append(precision_score(y_test, pred, average=None))
  print(recall_score(y_test, pred, average=None))
  print(precision_score(y_test, pred, average=None))

12
weights:  [[0.62922297 2.43464052]
 [0.74351297 1.52663934]
 [0.52650177 9.93333333]
 [0.82777778 1.26271186]
 [0.68537259 1.84863524]
 [0.64952049 2.17201166]
 [1.03185596 0.97005208]
 [0.54498903 6.05691057]
 [0.65696649 2.09269663]
 [0.59409888 3.15677966]
 [0.68726937 1.83497537]
 [0.57841615 3.68811881]]
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


10/10 [==============================] - 283s 28s/step - loss: 0.6908 - accuracy: 0.0646 - val_loss: 0.6311 - val_accuracy: 0.0638
Epoch 2/5
10/10 [==============================] - 299s 30s/step - loss: 0.6265 - accuracy: 0.1023 - val_loss: 0.6060 - val_accuracy: 0.0772
Epoch 3/5
10/10 [==============================] - 291s 29s/step - loss: 0.5873 - accuracy: 0.1300 - val_loss: 0.5843 - val_accuracy: 0.1980
Epoch 4/5
10/10 [==============================] - 295s 30s/step - loss: 0.5613 - accuracy: 0.2307 - val_loss: 0.5739 - val_accuracy: 0.1980
Epoch 5/5
10/10 [==============================] - 288s 29s/step - loss: 0.5357 - accuracy: 0.2416 - val_loss: 0.5785 - val_accuracy: 0.1846
[0.53703704 0.67708333 0.54545455 0.4587156  0.48717949 0.59701493
 0.63354037 0.75       0.73015873 0.84615385 0.77777778 0.78787879]
[0.25663717 0.51181102 0.06451613 0.43478261 0.30894309 0.33898305
 0.68       0.13274336 0.40707965 0.30275229 0.55752212 0.22033898]
12
weights:  [[0.62922297 2.4346405

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


10/10 [==============================] - 301s 30s/step - loss: 0.6805 - accuracy: 0.0805 - val_loss: 0.6392 - val_accuracy: 0.0369
Epoch 2/5
10/10 [==============================] - 243s 25s/step - loss: 0.6232 - accuracy: 0.0554 - val_loss: 0.6152 - val_accuracy: 0.0436
Epoch 3/5
10/10 [==============================] - 242s 24s/step - loss: 0.5830 - accuracy: 0.1426 - val_loss: 0.6061 - val_accuracy: 0.1141
Epoch 4/5
10/10 [==============================] - 241s 24s/step - loss: 0.5536 - accuracy: 0.1678 - val_loss: 0.5938 - val_accuracy: 0.1275
Epoch 5/5
10/10 [==============================] - 241s 24s/step - loss: 0.5247 - accuracy: 0.2122 - val_loss: 0.5786 - val_accuracy: 0.1980
[0.53333333 0.67676768 0.78571429 0.6875     0.54054054 0.64615385
 0.59863946 0.63636364 0.71428571 0.84444444 0.78947368 0.675     ]
[0.30769231 0.57758621 0.14285714 0.4502924  0.3125     0.43298969
 0.68217054 0.17948718 0.56122449 0.39583333 0.56074766 0.29032258]
12
weights:  [[0.62922297 2.4346405

...

10/10 [==============================] - 229s 23s/step - loss: 0.6733 - accuracy: 0.0747 - val_loss: 0.6408 - val_accuracy: 0.0738
Epoch 2/5
10/10 [==============================] - 226s 23s/step - loss: 0.6112 - accuracy: 0.1057 - val_loss: 0.6153 - val_accuracy: 0.1678
Epoch 3/5
10/10 [==============================] - 226s 23s/step - loss: 0.5764 - accuracy: 0.2139 - val_loss: 0.5850 - val_accuracy: 0.1879
Epoch 4/5
10/10 [==============================] - 229s 23s/step - loss: 0.5510 - accuracy: 0.1980 - val_loss: 0.5741 - val_accuracy: 0.1376
Epoch 5/5
10/10 [==============================] - 230s 23s/step - loss: 0.5271 - accuracy: 0.2416 - val_loss: 0.5660 - val_accuracy: 0.1779
[0.58823529 0.67647059 0.92307692 0.72131148 0.6627907  0.72463768
 0.83552632 0.76923077 0.8028169  0.75925926 0.81176471 0.69565217]
[0.36363636 0.52272727 0.12765957 0.49717514 0.33928571 0.40983607
 0.67914439 0.22988506 0.51351351 0.36607143 0.56557377 0.32323232]
12
weights:  [[0.62922297 2.4346405

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


10/10 [==============================] - 235s 23s/step - loss: 0.6722 - accuracy: 0.0713 - val_loss: 0.6575 - val_accuracy: 0.1141
Epoch 2/5
10/10 [==============================] - 236s 24s/step - loss: 0.6110 - accuracy: 0.1174 - val_loss: 0.6090 - val_accuracy: 0.1510
Epoch 3/5
10/10 [==============================] - 235s 24s/step - loss: 0.5703 - accuracy: 0.1636 - val_loss: 0.6002 - val_accuracy: 0.2013
Epoch 4/5
10/10 [==============================] - 235s 24s/step - loss: 0.5418 - accuracy: 0.2030 - val_loss: 0.5776 - val_accuracy: 0.2215
Epoch 5/5
10/10 [==============================] - 242s 24s/step - loss: 0.5125 - accuracy: 0.2240 - val_loss: 0.5711 - val_accuracy: 0.1779
[0.67857143 0.75789474 0.80952381 0.58333333 0.71604938 0.71428571
 0.74545455 0.76       0.81428571 0.76086957 0.86746988 0.61538462]
[0.296875   0.52554745 0.25       0.54609929 0.3452381  0.48672566
 0.73652695 0.19587629 0.45967742 0.3125     0.6        0.22429907]
12
weights:  [[0.62922297 2.4346405

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


10/10 [==============================] - 244s 24s/step - loss: 0.6646 - accuracy: 0.0671 - val_loss: 0.6676 - val_accuracy: 0.1141
Epoch 2/5
10/10 [==============================] - 240s 24s/step - loss: 0.6007 - accuracy: 0.1216 - val_loss: 0.6358 - val_accuracy: 0.1309
Epoch 3/5
10/10 [==============================] - 239s 24s/step - loss: 0.5610 - accuracy: 0.1762 - val_loss: 0.6247 - val_accuracy: 0.1611
Epoch 4/5
10/10 [==============================] - 239s 24s/step - loss: 0.5372 - accuracy: 0.2332 - val_loss: 0.6205 - val_accuracy: 0.1342
Epoch 5/5
10/10 [==============================] - 245s 25s/step - loss: 0.5059 - accuracy: 0.2525 - val_loss: 0.6166 - val_accuracy: 0.1577
[0.66176471 0.73958333 0.4375     0.64347826 0.60714286 0.75384615
 0.76223776 0.63333333 0.69333333 0.63461538 0.80246914 0.75      ]
[0.375      0.51824818 0.09459459 0.45962733 0.34931507 0.37984496
 0.66060606 0.22891566 0.44444444 0.33333333 0.55084746 0.28448276]


In [ ]:
#cybersecurity GloVe with weights
recall2 = []
prec2 = []
for i in range(12):
  avg = (recall[0][i] + recall[1][i] + recall[2][i] + recall[3][i] + recall[4][i]) / 5
  recall2.append(avg)
  avg = (prec[0][i] + prec[1][i] + prec[2][i] + prec[3][i] + prec[4][i]) / 5
  prec2.append(avg)

print(recall2)
print(prec2)
rec = 0
pre = 0
for i in range(12):
  rec = rec + recall2[i]
  pre = pre + prec2[i]
rec = rec / 12
pre = pre / 12
print(rec)
print(pre)

[0.5997883597883598, 0.7055599337023486, 0.7002539127539127, 0.6188677331886019, 0.6027405930506706, 0.6871876641636538, 0.7150796903869804, 0.7097855477855478, 0.7509760786943886, 0.7690684999380651, 0.8097910366382397, 0.7047831154352894]
[0.31996816789405286, 0.5311840247367655, 0.1359254881904161, 0.4775953535144194, 0.3310563934895709, 0.40967588642000036, 0.6876895868752484, 0.19338151022013966, 0.4771879026252829, 0.3420980777632153, 0.5669382031128258, 0.2685351421939165]
0.6978235137938382
0.3951029780863211


In [ ]:
#pretrained GloVe with weights
recall2 = []
prec2 = []
for i in range(12):
  avg = (recall[0][i] + recall[1][i] + recall[2][i] + recall[3][i] + recall[4][i]) / 5
  recall2.append(avg)
  avg = (prec[0][i] + prec[1][i] + prec[2][i] + prec[3][i] + prec[4][i]) / 5
  prec2.append(avg)

print(recall2)
print(prec2)
rec = 0
pre = 0
for i in range(12):
  rec = rec + recall2[i]
  pre = pre + prec2[i]
rec = rec / 12
pre = pre / 12
print(rec)
print(pre)

[0.5954186118892002, 0.6753723770209838, 0.5635864135864136, 0.6555197318914321, 0.7131440154695968, 0.6923515420335666, 0.639887310575444, 0.6852354312354312, 0.7232293965815094, 0.7660609438870308, 0.779844169707151, 0.7346442687747036]
[0.30020835253789585, 0.5021359564862957, 0.12145580064641123, 0.47361055709061883, 0.34000028575032426, 0.39279920003888225, 0.6691272801654109, 0.170226857066413, 0.4441035632017602, 0.31960641872228607, 0.535476680221424, 0.27340438071479234]
0.6853578510543721
0.3785129443868762


In [ ]:
#pretrained GloVe without weights
recall2 = []
prec2 = []
for i in range(12):
  avg = (recall[0][i] + recall[1][i] + recall[2][i] + recall[3][i] + recall[4][i]) / 5
  recall2.append(avg)
  avg = (prec[0][i] + prec[1][i] + prec[2][i] + prec[3][i] + prec[4][i]) / 5
  prec2.append(avg)

print(recall2)
print(prec2)
rec = 0
pre = 0
for i in range(12):
  rec = rec + recall2[i]
  pre = pre + prec2[i]
rec = rec / 12
pre = pre / 12
print(rec)
print(pre)

[0.007037037037037036, 0.4014867717421898, 0.0, 0.2456701659737095, 0.07295169349432915, 0.2508039397332064, 0.7024866176647804, 0.00909090909090909, 0.281458122472207, 0.08872414220240306, 0.522458115917402, 0.030164690382081683]
[0.09166666666666666, 0.5783795484147655, 0.0, 0.5809535515730568, 0.44833333333333336, 0.5160994397759103, 0.6520034520536349, 0.1, 0.5622987805956792, 0.3866666666666666, 0.6446387430816669, 0.38]
0.2176943504758546
0.41175334851344836
